In [1]:
import pandas as pd
import numpy as np
from six.moves import xrange
import math
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle
import time
from scipy.stats import f_oneway
from scipy.stats import pearsonr

In [2]:
time.ctime().replace(' ', '_')

'Fri_Mar_15_14:58:48_2019'

In [3]:
def ability_level_mapper(data, groups=None, col='front', how='naive', n_level=19, 
                         invert=True, parameters=None, imbalanced_data=False, target_col_name='performance'):
    # the raw data is divided into groups according to its exc_num, ability levels are calculated respectively
    # how: 1 is mapping without any other processing, called 'naive'
    origin = data.copy()
    # if target col is imbalanced, here we seperate it in parts with different scale value
    if parameters is not None:
        col, how, n_level, invert, v_max, v_min, imbalanced_data = parameters
        
    if imbalanced_data is not False:
        
        if imbalanced_data is True:
            origin['cnt'] = 1
            origin = origin.sort_values(by=col)
            origin = origin.reset_index()
            origin['cnt'] = origin['cnt'].cumsum()
            while(1):
                total = len(origin)
                each = int(total/(n_level+1))
                imbalanced_data = []
                for i in xrange(n_level):
                    v = origin.loc[(i+1)*each, col]
                    if v in imbalanced_data:
                        n_level -= 1
                        continue
                    else:
                        imbalanced_data.append(v)
                origin = origin.drop(['cnt'], axis=1)
                break
                    
        if type(imbalanced_data) is list:
            assert n_level==len(imbalanced_data), 'false length of imbalanced_data'
            # elements in inbalaced_data are ordered increasely
            origin[target_col_name] = 0
            for i,v in enumerate(imbalanced_data):
                if invert:
                    origin.loc[(origin.performance==0) & (origin[col]<=v), target_col_name] = n_level-i+1
                else:
                    origin.loc[(origin.performance==0) & (origin[col]<=v), target_col_name] = i+1
            if invert:
                origin.loc[origin.performance==0, target_col_name] = 1
            else:
                origin.loc[origin.performance==0, target_col_name] = n_level+1
                
        parameters = (col, how, n_level, invert, None, None, imbalanced_data)
        if 'index' in origin.columns:
            origin = origin.sort_values(by=['index'])
            origin = origin.set_index(['index'])
        return origin, parameters
            
    if parameters is not None:
        col, how, n_level, invert, v_max, v_min, imbalanced_data = parameters
        interval = (v_max- v_min)/n_level
        assert interval!=0, 'zero dividend'
        origin[target_col_name] = (origin[col]-v_min)/interval
        
        origin[target_col_name] = origin[target_col_name].astype(int)
        if invert:
            origin[target_col_name] = n_level-origin[target_col_name]+1
        else:
            origin[target_col_name] = origin[target_col_name]+1
        
        return origin, parameters
        
    if groups == None:
        v_max = origin[col].max()
        v_min = origin[col].min()
        
        interval = (v_max- v_min)/n_level
        assert interval!=0, 'zero dividend'
        origin[target_col_name] = (origin[col]-v_min)/interval
        
        origin[target_col_name] = origin[target_col_name].astype(int)
        if invert:
            origin[target_col_name] = n_level-origin[target_col_name]+1
        else:
            origin[target_col_name] = origin[target_col_name]+1
        
        parameters = (col, how, n_level, invert, v_max, v_min )
        return origin, parameters
    
    if how == 1 or how=='naive':
        tmp = origin[col]
        for index, group in groups:
            
            v_max = group[col].max()
            v_min = group[col].min()
            
            interval = (v_max-v_min)/n_level
            
            assert interval!=0, 'zero dividend'
            
            origin.loc[index, col] = (origin.loc[index, col]-v_min)/interval
        origin[col] = origin[col].astype(int)
        if invert:
            origin[target_col_name] = n_level-origin[col]+1
            
        else:
            origin[target_col_name] = origin[target_col_name]+1
        origin[col] = tmp
        parameters = (col, how, n_level, invert, v_max, v_min, False )
        
        return origin, parameters

def cMGRM(theta, alpha=1.5, b=0.5, c=0):
    # MGRM cumulative probability
    return math.e**(alpha*(theta-b+c))/(1+math.e**(alpha*(theta-b+c)))

def cMGRM_df(data, c):
    if 'gamma' in data.columns:
        return math.e**(data['alpha']*(data['theta']-data['b']+data['gamma']*c))/ \
            (1+math.e**(data['alpha']*(data['theta']-data['b']+data['gamma']*c)))
    
    if 'm' in data.columns:
        return math.e**(data['alpha']*(data['m']*data['theta']-data['b']+c))/ \
            (1+math.e**(data['alpha']*(data['m']*data['theta']-data['b']+c)))
    return math.e**(data['alpha']*(data['theta']-data['b']+c))/ \
                (1+math.e**(data['alpha']*(data['theta']-data['b']+c)))

def cMGRM_user_df(data, theta, c):
    # Matrix operation
    return math.e**(data['alpha']*(theta-data['b']+c))/(1+math.e**(data['alpha']*(theta-data['b']+c)))

def cMGRM_exc_df(data, alpha, b, c):
    # Matrix operation
    return math.e**(alpha*(data['theta']-b+c))/(1+math.e**(alpha*(data['theta']-b+c)))

def model(theta, alpha, b, cl=[1,0,-0.6], start=1):
    sum = start*1
    for c in cl:
        sum += cMGRM(theta, alpha, b, c)
    return sum

def model_df(data, cl, start=1):
    sum = pd.Series(np.zeros(len(data)))
    sum += start*1
#     print(data)
    for c in cl:
        sum+= cMGRM_df(data, c)
#         print(c)
#         print(cMGRM_df(data, c))
#     stop
#         print(sum)
        assert not sum.isnull().values.any(), 'na exists'
    return sum

def model_user_df(data, theta, cl, start=1):
    sum = pd.Series(np.zeros(len(data)))
    sum += start*1
    for c in cl:
        sum += cMGRM_user_df(data, theta, c)
    return sum

def model_exc_df(data, alpha, b, cl, start=1):
    sum = pd.Series(np.zeros(len(data)))
    sum += start*1
    for c in cl:
        sum += cMGRM_exc_df(data, alpha, b, c)
        
    return sum

def partial_derivative_model_alpha2_b2_df(data, alpha, b, cl):
    # partial derivative of model to alpha
    
    g_alpha = 0
    g_b = 0
    
    
    for c in cl:
        curr = cMGRM_exc_df(data, alpha, b, c)*(1-cMGRM_exc_df(data, alpha, b, c))*(-data['theta']+b-c)
        g_alpha += curr
        curr = cMGRM_exc_df(data, alpha, b, c)*(1-cMGRM_exc_df(data, alpha, b, c))*(alpha)
        g_b += curr
        
    return g_alpha, g_b

def partial_derivative_model_alpha2_b2_df2(data, cl, regular_term=True, zoom_factor=0.1, 
                                           max_thres=1000, verbose=False):
    # partial derivative of model to alpha
    
    g_alpha = 0
    g_b = 0
    g_gamma = 0
    g_m = 0
    regular_term_alpha = 0
    regular_term_b = 0
    
    for c in cl:
        if 'gamma' in data.columns:
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*(data['theta']-data['b']+data['gamma']*c)
            g_alpha += curr
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*data['alpha']*c
            g_gamma += curr
        elif 'm' in data.columns:
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*(data['m']*data['theta']-data['b']+c)
            g_alpha += curr
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*data['alpha']*data['theta']
            g_m += curr
        else:
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*(data['theta']-data['b']+c)
            g_alpha += curr
            
        curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*(-data['alpha'])
        g_b += curr
#     print(g_b)
#     assert g_b.mean()<0.05, 'too small partial derivative'
    if regular_term:
        for c in cl:
            #regular_term_alpha += zoom_factor*data['alpha']*(data['theta']-data['b']+c)**2  
            regular_term_alpha -= zoom_factor*data['alpha']
            #regular_term_b -= zoom_factor*data['alpha']**2*(data['theta']-data['b']+c)
            regular_term_b -= zoom_factor*data['b']
                 
                #+zoom_factor*(data['theta']-data['b']+c)**-3
            
    if verbose:
        print('----------partial_derivative_model_alpha2_b2_df2----------')
        print('derivative of alpha: ', g_alpha)
        if regular_term:
            print('regular term of alpha: \n')
            print(regular_term_alpha)
            
        print('derivative of b: ', g_b)
        if regular_term:
            print('regular term of b: \n')
            print(regular_term_b)
            
#         print('derivative of gamma:', g_gamma)
            
    g_alpha += regular_term_alpha
    g_b += regular_term_b
    g_alpha *= zoom_factor
    g_b *= zoom_factor
    
    g_alpha[g_alpha>max_thres] = max_thres
    g_alpha[g_alpha<-max_thres] = -max_thres
    g_b.loc[g_b>max_thres] = max_thres
    g_b.loc[g_b<-max_thres] = -max_thres
    
    if 'gamma' in data.columns:
        g_gamma.loc[g_gamma>max_thres] = max_thres
        g_gamma.loc[g_gamma<-max_thres] = -max_thres
        return g_alpha, g_b, g_gamma
    if 'm' in data.columns:
        g_m.loc[g_m>max_thres] = max_thres
        g_m.loc[g_m<-max_thres] = -max_thres
        return g_alpha, g_b, g_m
    
    return g_alpha, g_b



def partial_derivative_model_theta2_df(data, theta, cl):
    # partial derivative of model to theta
    g_theta = 0
    for c in cl:
        curr = cMGRM_user_df(data,theta, c)*(1-cMGRM_user_df(data,theta, c))*(-data['alpha'])
        g_theta += curr
     
    return g_theta

def partial_derivative_model_theta2_df2(data, cl, regular_term=True, zoom_factor=0.1, max_thres=1000,
                                       verbose=False):
    # partial derivative of model to theta
    g_theta = 0
    regular_term_theta = 0
    
    for c in cl:
        if 'm' in data.columns:
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*data['alpha']*data['m']
        else:
            curr = cMGRM_df(data, c)*(1-cMGRM_df(data, c))*(data['alpha'])
        g_theta += curr
#     print(g_theta)
#     assert g_theta.mean()>0.1, 'too small partial derivative'
        
        
    if regular_term:
        for c in cl:
            # regular_term_theta += zoom_factor*data['alpha']**2*(data['theta']-data['b']+c) 
            # - zoom_factor*(data['theta']-data['b']+c)**-3
            regular_term_theta -= zoom_factor*data['theta']
    
    if verbose:
        print('--------partial_derivative_model_theta2_df2----------')
        print('derivative of theta: \n')
        print(g_theta)
        if regular_term:
            print('regular term of theta: \n')
            print(regular_term_theta)
            
    
    g_theta += regular_term_theta
    g_theta *= zoom_factor       
    g_theta[g_theta>max_thres] = max_thres
    g_theta[g_theta<-max_thres] = -max_thres
    if zoom_factor >10000000:
        g_theta = 1
    return g_theta

def constraint_theta(data, parameters, cl, item_by=['difficulty'], 
                     user_by=['uid', 'day', 'exc_num', 'exc_times'], 
                     zoom_factor=0.01, max_thres=500):
    """
    this function is used to calculate partial derivative for theta in the fomula 
    min sum((theta-b+(c_k+c_(k+1))/2)**2)
    """
    
    uid_p, exc_p = parameters
    # combine parameter and data
    tmp = data.merge(exc_p, on=item_by, how='left')
    tmp = tmp.merge(uid_p, on=user_by, how='left')
#     print(tmp.head())
    # calc paritial derivative
    if 'gamma' in exc_p.columns:
        tmp['g_theta'] =  tmp['theta']-tmp['b']+tmp['gamma']*tmp['cl_term']
    else:
        tmp['g_theta'] =  tmp['theta']-tmp['b']+tmp['cl_term']
    gradient_theta = tmp[user_by+['g_theta']].groupby(user_by).sum()*2
    gradient_theta = gradient_theta.reset_index()
    gradient_theta.columns = user_by+['theta']
    gradient_theta['theta'] *= zoom_factor
    
    gradient_theta.loc[gradient_theta['theta']>max_thres, 'theta'] = max_thres
    gradient_theta.loc[gradient_theta['theta']<-max_thres, 'theta'] = -max_thres
    
    return gradient_theta

def constraint_b(data, parameters, cl, item_by=['difficulty'], 
                     user_by=['uid', 'day', 'exc_num', 'exc_times'], 
                 zoom_factor=0.01, zoom_factor_gamma=0.001, max_thres=500):
    """
    this function is used to calculate partial derivative for theta in the fomula 
    min sum((theta-b+(c_k+c_(k+1))/2)**2)
    """
    
    uid_p, exc_p = parameters
    # combine parameter and data
    tmp = data.merge(exc_p, on=item_by, how='left')
    tmp = tmp.merge(uid_p, on=user_by, how='left')
#     print(tmp.head())
    # calc paritial derivative
    if 'gamma' in exc_p.columns:
        tmp['g_b'] =  -tmp['theta']+tmp['b']-tmp['gamma']*tmp['cl_term']
        tmp['g_gamma'] = (tmp['theta']-tmp['b']+tmp['gamma']*tmp['cl_term'])*tmp['cl_term']
        gradient_b = tmp[item_by+['g_b', 'g_gamma']].groupby(item_by).sum()*2
        
    else:
        tmp['g_b'] =  -tmp['theta']+tmp['b']-tmp['cl_term']
        gradient_b = tmp[item_by+['g_b']].groupby(item_by).sum()*2
        
        
    gradient_b = gradient_b.reset_index()
    
    if 'gamma' in exc_p.columns:
        gradient_b.columns = item_by+['b', 'gamma']
        gradient_b['gamma'] *= zoom_factor_gamma
    else:
        gradient_b.columns = item_by+['b']
        
    gradient_b['b'] *= zoom_factor
    gradient_b['alpha'] = 0
    
    if 'gamma' in exc_p.columns:
        gradient_b.loc[gradient_b['gamma']>max_thres, 'gamma'] = max_thres
        gradient_b.loc[gradient_b['gamma']<-max_thres, 'gamma'] = -max_thres
   
    gradient_b.loc[gradient_b['b']>max_thres, 'b'] = max_thres
    gradient_b.loc[gradient_b['b']<-max_thres, 'b'] = -max_thres
    
    return gradient_b


def loss(theta, alpha, b, cl, y):
    return (model(theta, alpha, b, cl)-y)**2

def sum_loss(data, parameters, cl, theta_by=['uid','day','exc_num','exc_times'], item_by=['difficulty'], 
             verbose=False):
    uid_p, exc_p = parameters
    tmp = data.copy()
    tmp = tmp.merge(uid_p, on=theta_by)
    tmp = tmp.merge(exc_p, on=item_by)
    exp = model_df(tmp, cl)
    tmp['diff'] = exp-tmp['performance']
    
    if verbose:
        sns.set_style('whitegrid')
        f, ax= plt.subplots(figsize = (14, 10))
        ax = sns.boxplot(x='exc_num', y='diff', data=tmp)
        plt.show()

        print(tmp['diff'].describe())
        print('\n\n')
        print('max')
        mx = tmp[tmp['diff']==tmp['diff'].max()]
        mx = mx.reset_index(drop=True)
        if len(mx)>1:
            mx = mx.loc[0,]
        print(mx)
        print('\n')
        print(tmp.set_index(theta_by).loc[mx[theta_by].values.tolist()].describe())
        print('\n\n')
        print('min')
        mi = tmp[tmp['diff']==tmp['diff'].min()]
        mi = mi.reset_index(drop=True)
        if len(mi)>1:
            mi = mi.loc[0, ]

        print(mi)
        print('\n')
        print(tmp.set_index(theta_by).loc[mi[theta_by].values.tolist()].describe())

        print('\n\n')
        print('std')
        std = tmp.groupby(theta_by)['diff'].std().reset_index()
        sns.set_style('whitegrid')
        f, ax= plt.subplots(figsize = (14, 10))
        ax = sns.boxplot(x='exc_num', y='diff', data=std)
        plt.show()

        print('\n\n')
        print('std theta')
        theta_std = tmp.groupby(['uid', 'day', 'exc_num'])['theta'].std().reset_index()
        days = theta_std['day'].unique()
        for day in days:
            print(day)
            sns.set_style('whitegrid')
            f, ax= plt.subplots(figsize = (14, 10))
            ax = sns.boxplot(x='uid', y='theta', data=theta_std[theta_std['day']==day])
            plt.show()
            print('\n')

        print('\n\n')
        print('mean theta')
        theta_mean = tmp.groupby(['uid', 'day', 'exc_num'])['theta'].mean().reset_index()
        days = theta_mean['day'].unique()
        sns.set_style('whitegrid')
        f, ax= plt.subplots(figsize = (14, 10))
        ax = sns.boxplot(x='uid', y='theta', hue='day', data=theta_mean)
        plt.show()
        print('\n')
    
    tmp['diff'] = tmp['diff']**2
    s = np.sum(tmp['diff'])
    mean = np.mean(tmp['diff'])
    
    del tmp
    return s, mean

def calc_gradient_user(data, parameters, cl, theta_by=['uid','day','exc_num','exc_times'], 
                       item_by=['difficulty'], zoom_factor=0.1, regular_term=True, verbose=False):
    # calc sum loss for a certain user, so we select all loss by this user
    gradient_theta = pd.DataFrame()
    uid_p, exc_p = parameters
    
    # groups will be seperate by ability definitions
#     groups = data.groupby(theta_by)
#     check_theta = uid_p.set_index(theta_by)
    assert len(data) == len(data.drop_duplicates()), \
    'maybe the expected value for each difficulty in each exercise repetition should be calculated first'
    
    gradient_theta = data.copy()
    gradient_theta = gradient_theta.merge(uid_p, on=theta_by)
    gradient_theta = gradient_theta.merge(exc_p, on=item_by)
    gradient_theta['ev'] = model_df(gradient_theta, cl)
    gradient_theta['g_theta'] = partial_derivative_model_theta2_df2(gradient_theta, cl, zoom_factor=zoom_factor,
                                                                    regular_term=regular_term,
                                                                   verbose=verbose)
    gradient_theta['g_theta'] = (gradient_theta['ev']-gradient_theta['performance'])*gradient_theta['g_theta']
    gradient_theta = gradient_theta[theta_by+['g_theta']].groupby(theta_by).sum()*2
    gradient_theta = gradient_theta.reset_index()
    gradient_theta.columns = theta_by+['theta']
    
#     for index, group in groups:
#         # calculate the expected value with current estimated theta for each item
#         # here item means the moment in one exercise with some difficulty
#         theta = check_theta.loc[index, 'theta']
#         difficulties = group['difficulty'].unique()
#         exc_parameters = exc_p.set_index(item_by).loc[difficulties].reset_index()
#         expected_value = model_user_df(exc_parameters, theta=theta, cl=cl)
#         y = group.loc[:, 'performance'].reset_index(drop=True)
#         difficulties2 = group.loc[:, 'difficulty'].tolist()
#         assert difficulties.tolist()==difficulties2, 'false alignment between expected value and average value'
#         diff = expected_value-y
#         g_theta = partial_derivative_model_theta2_df(exc_parameters, theta, cl)
#         g_theta = 2*np.sum(diff*g_theta)


#         curr = pd.DataFrame(np.array(list(index)+[g_theta]).reshape(1,-1), columns=theta_by+['theta'])
#         gradient_theta = pd.concat([gradient_theta, curr])
        
     
#     gradient_theta = gradient_theta.reset_index(drop=True)
#     del check_theta
#     del groups
#     print(gradient_theta)
    return gradient_theta

def calc_gradient_exc(data, parameters, cl, theta_by=['uid','day','exc_num','exc_times'], 
                      item_by=['difficulty'], zoom_factor=0.1, regular_term=True, verbose=False):
    # calc sum loss for a certain exercise, so we select all loss by this exercise
    gradient_alpha_b = pd.DataFrame()
    uid_p, exc_p = parameters
    assert len(data) == len(data.drop_duplicates()), \
    'maybe the expected value for each difficulty in each exercise repetition should be calculated first'
    
    gradient_alpha_b = data.copy()
    gradient_alpha_b = gradient_alpha_b.merge(uid_p, on=theta_by)
    gradient_alpha_b = gradient_alpha_b.merge(exc_p, on=item_by)
#     print(1)
#     print(gradient_alpha_b)
    gradient_alpha_b['ev'] = model_df(gradient_alpha_b, cl)
    if 'gamma' in exc_p.columns:
        gradient_alpha_b['g_alpha'], gradient_alpha_b['g_b'], gradient_alpha_b['g_gamma'] = \
            partial_derivative_model_alpha2_b2_df2( \
            gradient_alpha_b, cl, zoom_factor=zoom_factor, regular_term=regular_term, verbose=verbose)
    elif 'm' in exc_p.columns:
        gradient_alpha_b['g_alpha'], gradient_alpha_b['g_b'], gradient_alpha_b['g_m'] = \
            partial_derivative_model_alpha2_b2_df2( \
            gradient_alpha_b, cl, zoom_factor=zoom_factor, regular_term=regular_term, verbose=verbose)
    else:
        gradient_alpha_b['g_alpha'], gradient_alpha_b['g_b'] = partial_derivative_model_alpha2_b2_df2( \
            gradient_alpha_b, cl, zoom_factor=zoom_factor, regular_term=regular_term, verbose=verbose)

    # The sign of the gradient is decided by partial derivative model
    gradient_alpha_b['g_alpha'] = (gradient_alpha_b['ev']-gradient_alpha_b['performance']) \
        *gradient_alpha_b['g_alpha']
    
    gradient_alpha_b['g_b'] = (gradient_alpha_b['ev']-gradient_alpha_b['performance'])*gradient_alpha_b['g_b']
    
    if 'gamma' in exc_p.columns:
        gradient_alpha_b['g_gamma'] = (gradient_alpha_b['ev']-gradient_alpha_b['performance'])* \
            gradient_alpha_b['g_gamma']
    if 'm' in exc_p.columns:
        gradient_alpha_b['m'] = (gradient_alpha_b['ev']-gradient_alpha_b['performance'])* \
            gradient_alpha_b['m']
        
    gradient_alpha_b['diff'] = gradient_alpha_b['ev']-gradient_alpha_b['performance']
#     print(find_segment(gradient_alpha_b, ['uid', 'day', 'exc_num', 'exc_times'], [1, 3, 3.2, 1]))
#     print(find_segment(gradient_alpha_b, ['uid', 'day', 'exc_num', 'exc_times'], [1, 5, 3.2, 2]))
#     stop()
    
    if 'gamma' in exc_p.columns:
        gradient_alpha_b = gradient_alpha_b[item_by+['g_alpha', 'g_b', 'g_gamma']].groupby(item_by).sum()*2
    elif 'm' in exc_p.columns:
        gradient_alpha_b = gradient_alpha_b[item_by+['g_alpha', 'g_b', 'g_m']].groupby(item_by).sum()*2
    else:
        gradient_alpha_b = gradient_alpha_b[item_by+['g_alpha', 'g_b']].groupby(item_by).sum()*2
        
        
    gradient_alpha_b = gradient_alpha_b.reset_index()
    
    
    if 'gamma' in exc_p.columns:
        gradient_alpha_b.columns = item_by+['alpha', 'b', 'gamma']
    elif 'm' in exc_p.columns:
        gradient_alpha_b.columns = item_by+['alpha', 'b', 'm']
    else:
        gradient_alpha_b.columns = item_by+['alpha', 'b']
        
        
    return gradient_alpha_b

def calc_ANOVA(df, pos1, pos2, target_col, by=['uid', 'day', 'exc_num']):
    tmp = df.set_index(by)
    group1 = tmp.loc[tuple(pos1),target_col]#.reset_index()
    group2 = tmp.loc[tuple(pos2), target_col]#.reset_index()
    
    f_value, p_value = f_oneway(group1, group2)
    return f_value, p_value
    
    

def combine_data_cl(data, cl, del_min_max=True):
    """
    del_min_max:    if True, delete performance with 1 and len(cl), then they will not be 
                    considered for optimization
    """
    if del_min_max:
        data = data[data['performance']!=1]
        data = data[data['performance']!=len(cl)+1]
    else:
        data['cl_term'] = 0
        data.loc[(data.performance==1) | (data.performance==len(cl)+1), 'cl_term'] = np.nan
    for i in xrange(len(cl)-1):
        data.loc[data.performance==i+2, 'cl_term'] = (cl[i]+cl[i+1])/2
        
    return data

def calc_test_theta(df, cl, exc_p, user_by= ['uid', 'day', 'exc_num', 'exc_times'],
                    item_by=['difficulty'], how='practical', real_time=False, **params):
    """
    df:    input data, maybe online data
    """
    
    if how=='extremum':
        """
        this method only consider the moment status(ability)
        """
        cols = df.columns+['theta']
        df = df.merge(exc_p, on=item_by, how='left')
        df['theta'] = np.nan
        df.loc[df.performance==1, 'theta'] = -4
        df.loc[df.performance==len(cl), 'theta'] = 4
        tmp =  df.loc[(df.performance>1) & (df.performance<len(cl)), ['theta', 'b', 'performance']].reset_index()
        tmp['performance'] = tmp['performance'].astype(int)

        for i in xrange(len(cl)-1):
            tmp.loc[tmp.performance==i+2, 'theta'] = tmp.loc[tmp.performance==i+2, 'b']-(cl[i]+cl[i+1])/2

        tmp = tmp.set_index(['index'])
        df.loc[(df.performance>1) & (df.performance<len(cl)), 'theta'] = tmp['theta']
        return df
    elif how=='practical':
        """
        this method consider the historical data
        length:      how long data will be taken for the estimation
        """
        result = []
        df_tmp = pd.DataFrame()
        l = params['length']
        cadidates = np.linspace(-4, 4, 100)
        avg = df.groupby(user_by+item_by).mean()['performance'].reset_index()
        avg = avg.merge(exc_p, on=['difficulty'], how='left')
        avg['result'] = 0
        avg['min_loss'] = 10000
        for ca in cadidates:
            avg['theta'] = ca
            avg['exp'] = model_df(avg, cl)
            avg['loss'] = (avg['exp']-avg['performance'])**2
            avg['loss'] = avg.groupby(['uid', 'day', 'exc_num', 'exc_times'])['loss'].cumsum()
            avg.loc[avg.loss<avg.min_loss, 'result'] = ca
            avg.loc[avg.loss<avg.min_loss, 'min_loss'] = avg.loc[avg.loss<avg.min_loss, 'loss']
        
        df_tmp = avg[user_by+['result']]
        df_tmp = df_tmp.groupby(user_by).tail(1)
#         print(df_tmp)
        df_tmp.columns = user_by+['theta']
#         for index, group in avg.groupby(user_by):            
#             min_loss = 10000
#             theta = None
#             for ca in cadidates:
#                 group['theta'] = ca
#                 exp = model_df(avg, cl)
#                 loss = (exp-avg['performance'])**2
#                 loss = loss.sum()
                
#                 if loss<min_loss:
#                     min_loss = loss
#                     theta = ca
#             curr = pd.DataFrame(np.array(list(index)+[theta]).reshape(1,-1), columns=user_by+['theta'])
#             df_tmp = pd.concat([df_tmp, curr])
                    
#             result.append(theta)
        
        return df_tmp
#     elif how=='practical2':
        
        
        


def update(parameter, gradient, alpha=0.001):
    uid_p, exc_p = parameters
    uid_gradient, exc_gradient = gradient
    
    # calc average theta gradient by each user in each day
    uid_avg = uid_gradient.groupby(['uid', 'day']).mean()    
    # calc average alpha and b by each exercise
    exc_avg = exc_gradient.groupby(['exc_num']).mean()
    
    uid_step = uid_avg*alpha
    exc_step = exc_avg*alpha
    
    uid_result = pd.concat([uid_p.set_index(['uid', 'day']), uid_step], join='inner', axis=1)
    exc_result = pd.concat([exc_p.set_index(['exc_num']), exc_step], join='inner', axis=1)
    
    uid_result['theta'] -= uid_result['theta_g']
    exc_result['alpha'] -= exc_result['alpha_g']
    exc_result['b'] -= exc_result['b_g']
    
    return [uid_result.reset_index()[['uid', 'day', 'theta']], exc_result.reset_index()[['exc_num', 'alpha', 'b']]]

def update2(parameters, optimizer_uid, optimizer_exc, theta_by=['day','uid','exc_num','exc_times'],
           item_by=['difficulty'], a_lower_limit=10**-8):
    uid_p, exc_p = parameters
    uid_p = uid_p.set_index(theta_by) - optimizer_uid.set_index(theta_by)
    exc_p = exc_p.set_index(item_by) - optimizer_exc.set_index(item_by)
    uid_p.loc[uid_p.theta>10, 'theta'] = 10
    uid_p.loc[uid_p.theta<-10, 'theta'] = -10
    exc_p.loc[exc_p.alpha>10, 'alpha'] = 10
    exc_p.loc[exc_p.alpha<0, 'alpha'] = a_lower_limit
#     exc_p.loc[exc_p.m<0, 'm'] = a_lower_limit
#     exc_p.loc[exc_p.b>10, 'b'] = 10
#     exc_p.loc[exc_p.b<-10, 'b'] = -10
    return uid_p.reset_index(), exc_p.reset_index()


def merge_minority(data, by=['difficulty'],threshold=10):
    cnt = 1
    while cnt>0:
        cnt = 0
        for index, group in data.groupby(by):
            d = index
            if len(group)<threshold:
                if d<data['difficulty'].max()/2:
                    data.loc[data.difficulty==d, 'difficulty'] +=1
                else:
                    data.loc[data.difficulty==d, 'difficulty'] -=1
                cnt += 1
    return data       
    
def initial_parameters(data, uid_by=['uid', 'day', 'exc_num', 'exc_times'], 
                       exc_by=['difficulty'], with_gamma=True, with_m=True):
    uid_cols = uid_by+['theta']
    
    
    if with_gamma:
        exc_cols = exc_by+['alpha', 'b', 'gamma']
    elif with_m:
        exc_cols = exc_by+['alpha', 'b', 'm']
    else:
        exc_cols = exc_by+['alpha', 'b']
        
        
    uid_p = pd.DataFrame(np.array([]).reshape(0, len(uid_cols)), columns=uid_cols)
    exc_p = pd.DataFrame(np.array([]).reshape(0, len(exc_cols)), columns=exc_cols)
    
    # compare average score over all difficulty of different user
    avg_perf = data[['uid', 'performance']].groupby(['uid']).mean().reset_index()
    
    # transform average score in range of [-3, 3]
    max_perf = avg_perf['performance'].max()
    min_perf = avg_perf['performance'].min()
    interval = max_perf-min_perf
    
    avg_perf['theta'] = (avg_perf['performance']-min_perf)/interval*6-3
    avg_perf = avg_perf.drop_duplicates()
    
    tmp = data[['uid', 'day', 'exc_num', 'exc_times', 'performance']].groupby(uid_by) 
    
    for index,group in tmp:
        uid, _, _, _ = index
        
        curr = pd.DataFrame(np.array(list(index)+[0]).reshape(1, 5), 
                            columns=uid_cols)
        curr['theta'] = avg_perf.loc[avg_perf.uid==uid, 'theta'].tolist()[0]
        uid_p = pd.concat([uid_p, curr])
        
    
    #################################
    # compare average score over all user of different exercises
    avg_perf = data[['difficulty', 'performance']].groupby(exc_by).mean()
    
    
    # use average score to represent exercise difficulty, and transform them in range of [-2, 2]
    max_perf = avg_perf['performance'].max()
    min_perf = avg_perf['performance'].min()
    interval = max_perf-min_perf
    
    avg_perf['b'] = (avg_perf['performance']-min_perf)/interval*4-2
    avg_perf['b'] = avg_perf['b']*-1
    
    avg_perf = avg_perf.reset_index()
#     avg_perf['alpha'] = pd.Series(1.5*np.random.random_sample(len(avg_perf)))
    avg_perf['alpha'] = 1
    
    if with_gamma:
        avg_perf['gamma'] = pd.Series(1.5*np.random.random_sample(len(avg_perf)))
    if with_m:
        avg_perf['m'] = 1
    
    exc_p = avg_perf[exc_cols]
    uid_p = uid_p.reset_index(drop=True)
    exc_p = exc_p.reset_index(drop=True)
    
    return uid_p, exc_p
    
    
def select_exc(data, selected_excs=None, non_selected_excs=None, selected_diff=None,
               non_selected_diff=None, selected_uid=None, non_selected_uid=None, delete=None):
    data = data.set_index(['exc_num'])  
    if selected_excs is not None:
        data = data.loc[selected_excs]
    if non_selected_excs is not None:
        data = data.reset_index()
        for e in non_selected_excs:
            data = data.loc[data.exc_num!=e]
            
    data = data.reset_index()
    data = data.set_index(['difficulty'])
    if selected_diff is not None:
        data = data.loc[selected_diff]
    if non_selected_diff is not None:
        data = data.reset_index()
        for diff in non_selected_diff:
            data = data.loc[data.difficulty!=diff]
            
    data = data.reset_index()
    data = data.set_index(['uid'])       
    if selected_uid is not None:
        data = data.loc[selected_uid]
    if non_selected_uid is not None:
        data = data.reset_index()
        for uid in non_selected_uid:
            data = data.loc[data.uid!=uid]
    return data.reset_index()

def set_cl(n=19, start=-1, end=1, test_col_p=None):
    if test_col_p is None or test_col_p is False:
        if start<end:
            tmp = start
            start = end
            end = tmp
        return list(np.linspace(start, end, n))
    # if test column is segmented in different length, the c list should have the same distributiion
    else:
        cl = []
        lth = end-start
        # test_col_p denotes the percentage of each point in the whole scale
        # e.g. [0, 0.05, 0.10, ..., 1]
        l2 = test_col_p[-1]-test_col_p[0]
        
        
        for element in test_col_p:
            cl.append((element-test_col_p[0])*lth/l2+start)
        return cl.sort(reverse=True)

def learning_rate_optimizer(gradient, history, cols, how='SGD-M', **params):
    """
    history: previous monontums
    """
    if type(history) is not list:
        history = [history]
        
    if how is None:
        """
        input:
        err_thres:    used to check if it is necessary to random a learning rate
        eta:          learning rate
        grad_thres:   whose gradients are too small, and need a random learning rate
        mean_sqr_err: mean loss
        
        output:
        delta:        <DataFrame>
        """
        delta = gradient.copy()
        err_thres = params['err_thres']
        eta = params['eta']
        grad_thres = params['grad_thres']
        mean_sqr_err = params['mean_sqr_err']
        max_step = params['max_step']
        l = len(delta)
        
        assert err_thres>0 and grad_thres>0 and mean_sqr_err>0, 'all parameter should be larger than 0'
        
        for col in cols:
            mean_grad = gradient[col].mean()
            sum_grad = gradient[col].abs().sum()
            proportion = gradient[col].abs()/sum_grad
            delta[col] = gradient[col]*proportion*eta
            
#             if mean_sqr_err>err_thres:
#                 delta.loc[(delta[col]<grad_thres) & (delta[col]>0), col] = pd.Series(np.random.rand(l))
#                 delta.loc[(delta[col]>-grad_thres) & (delta[col]<0), col] = -1*pd.Series(np.random.rand(l))
                
            delta.loc[delta[col]>max_step, col] = max_step
            delta.loc[delta[col]<-max_step, col] = -max_step
        
        assert not delta.isnull().values.any(), 'na exists'
        
        return delta, history
        
    if how == 'SGD-M':
        v_t = 1
        gamma = params['gamma']
        eta = params['eta']
        max_step = params['max_step']
        m_t = gradient
        
        for col, hry in zip(cols, history):
            if len(hry)==0:
                m_t[col] = eta*gradient[col]

            else:
                pre_m = hry[-1]
                m_t[col] = gamma*pre_m[col] + eta*gradient[col]
            hry.append(m_t)
        
            m_t.loc[m_t[col]>max_step, col] = max_step
            m_t.loc[m_t[col]<-max_step, col] = -max_step
        return m_t, history
    
def generate_log(non_selected_cols, selected_cols, non_selected_diff, selected_diff, non_selected_uid,
                selected_uid):
    filename = time.ctime().replace(' ', '_')
    with open('../data/parameter/def_env.p', 'rb') as f:
        tmp = pickle.load(f)
        f.close()
        assert tmp is not None, 'env parameters are none'
    test_col, diff_def, n_class, group, if_del_outlier, if_merge_min, sampleing_rate = tmp
    with open('../data/parameter/'+filename, 'w') as f:
        f.write('test column: '+test_col)
        f.write('\n')
        f.write('difficulty: '+''.join(str(x)+',' for x in diff_def))
        f.write('\n')
        f.write('n_class: '+''.join(str(x)+',' for x in n_class))
        f.write('\n')
        f.write('group: '+''.join(str(x)+',' for x in group))
        f.write('\n')
        f.write('delete outlier in the first stage: '+str(if_del_outlier))
        f.write('\n')
        f.write('if merge minority: '+ str(if_merge_min))
        f.write('\n')
        f.write('sampling rate: '+str(sampleing_rate))
        f.write('\n')
        if selected_cols is not None:
            f.write('selected exercises: '+''.join(str(x)+',' for x in selected_cols))
            f.write('\n')
        if non_selected_cols is not None:
            f.write('non selected exercises: '+''.join(str(x)+',' for x in non_selected_cols))
            f.write('\n')
        if selected_diff is not None:
            f.write('selected difficulty: '+''.join(str(x)+',' for x in selected_diff))
            f.write('\n')
        if non_selected_diff is not None:
            f.write('non selected difficulty: '+''.join(str(x)+',' for x in non_selected_diff))
            f.write('\n')
        if selected_uid is not None:
            f.write('selected uid: '+''.join(str(x)+',' for x in selected_uid))
            f.write('\n')
        if non_selected_uid is not None:
            f.write('non selected uid: '+''.join(str(x)+',' for x in non_selected_uid))
            f.write('\n')
        f.write('\n\n')
        
    return filename

def generate_test(df, n=0, selected=None):
    """
    n: number of user that used as test data
    return:  test data, test uid 
    """
    if n == 0:
        return None, None
    elif selected is not None:
        test_num = []
        assert selected in df['uid'].unique().tolist(), 'invalid uid'
        test = pd.DataFrame()
        if selected is not list:
            selected = [selected]
        for uid in selected:
            tmp = df.loc[df.uid==uid, ]
            test = pd.concat([test, tmp])
        test_num = selected
    else:
        test_num = []
        test = pd.DataFrame()
        uid = df['uid'].unique()
        print(uid)
        for i in xrange(n):
            
            tmp_num = random.randint(1, len(uid))
            if tmp_num>9:
                tmp_num += 1
                tmp_num = uid[tmp_num-2]
            else:
                tmp_num = uid[tmp_num-1]
            test_num.append(tmp_num)
            tmp = df.loc[df.uid==tmp_num, ]
            test = pd.concat([test, tmp])
            
    return test.reset_index(drop=True), test_num

def write_log(mean_loss, sum_loss, filename):
    with open('../data/parameter/'+filename, 'a') as f:
        f.write('mean loss: '+str(mean_loss))
        f.write('\n')
        f.write('sum loss: '+str(sum_loss))
        f.write('\n\n')
        
def get_env_parameters():
    with open('../data/parameter/def_env.p', 'rb') as f:
        tmp = pickle.load(f)
        f.close()
        assert tmp is not None, 'env parameters are none'
    return tmp

def find_segment(df, cols, values):
    tmp = df.copy()
    for col, value in zip(cols, values):
        tmp = tmp[tmp[col]==value]
        
    return tmp

def get_test_col_name(test_col):
    if test_col == 'velocity':
        return 'velocity','velocity'
    elif test_col == 'vd_score':
        return 'score(velocity/deviation)', 'vd_score'
    elif test_col == 'deviation':
        return 'deviation','deviation'
    elif test_col == 'force':
        return 'force', 'force'
    elif test_col == 'tt_freq':
        return 'tremble frequency', 'tt_freq'
    elif test_col == 're_dev':
        return 'reciprocal deviation', 're_dev'
    elif test_col == 'inv_tt_freq':
        return 'inverted tremble frequency', 'inv_tt_freq'
    
def select_data(data, selected_excs=None, non_selected_excs=None, selected_uid=None,
                non_selected_uid=None):
    ret = None
    if selected_excs is not None:
        ret = pd.DataFrame()
        for exc in selected_excs:
            curr = data[data['exc_num']==exc]
            ret = pd.concat([ret, curr])
            
    if non_selected_excs is not None:
        if ret is not None:
            data = ret
        for exc in non_selected_excs:
            data = data[data['exc_num']!=exc]
        ret = data
        
    if selected_uid is not None:
        if ret is not None:
            data = ret
        ret = pd.DataFrame()
        for uid in selected_uid:
            curr = data[data['uid']==uid]
            ret = pd.concat([ret, curr])
            
    if non_selected_uid is not None:
        if ret is not None:
            data = ret
        for uid in non_selected_uid:
            data = data[data['uid']!=uid]
        ret = data
        
    return ret
                


In [4]:
all_data = pd.read_csv('../data/step2_expected_performance.csv')
expected_performance = all_data[['day', 'exc_num', 'uid', 'exc_times', 'difficulty','performance']]. \
    groupby(by=['day', 'exc_num', 'uid', 'exc_times', 'difficulty']).mean().reset_index()
print('exercises:', expected_performance['exc_num'].unique())
# expected_performance = expected_performance[expected_performance['day']!=1]
# selected_excs = [1.1, 2.1, 3.1]

with_gamma = False
with_m = True
selected_excs = None
non_selected_excs = [4.1, 4.2]
# [2.2, 2.3, 4.1, 4.2, 4.3]
selected_diff = None
non_selected_diff = None #[1, 2, 8 ,10]
test, test_uid = generate_test(expected_performance, 1, 4)
selected_uid = None
non_selected_uid = test_uid
expected_performance = select_exc(expected_performance, selected_excs=selected_excs,
                                  non_selected_excs=non_selected_excs, selected_diff=selected_diff,
                                 non_selected_diff=non_selected_diff, selected_uid=selected_uid,
                                 non_selected_uid=non_selected_uid)
print('user index:', expected_performance['uid'].unique())
print('# of course repetitions: ', len(expected_performance.groupby(['uid', 'day', 'exc_num', 'exc_times']).mean()))
# n_cls = 19
with open('../data/parameter/ability_mapper.p', 'rb') as f:
    mapping_parameters = pickle.load(f)
col, how, n_level, invert, v_max, v_min, imbalanced_data = mapping_parameters
cl = set_cl(n_level, test_col_p=imbalanced_data)
print('cl:', cl)
all_data = select_data(all_data, selected_excs=selected_excs, non_selected_excs=non_selected_excs, 
                       selected_uid=selected_uid, non_selected_uid=non_selected_uid)
test = select_data(test, selected_excs=selected_excs, non_selected_excs=non_selected_excs)
all_data = combine_data_cl(all_data, cl, del_min_max=True)
# print(all_data['exc_num'].unique())
assert len(all_data[all_data['cl_term']==0])==0, 'there is category that is not assigned to a performance'
assert not all_data.isnull().values.any(), 'there is na in the dataframe'
log_name = generate_log(non_selected_excs, selected_excs, non_selected_diff, selected_diff, 
                        non_selected_uid, selected_uid)

exercises: [ 1.1  1.5  2.1  2.2  3.1  3.2  4.1  4.2]
user index: [  1.   2.   5.   6.   7.   3.   8.  10.  11.]
# of course repetitions:  369
cl: [1.0, 0.88888888888888884, 0.77777777777777779, 0.66666666666666674, 0.55555555555555558, 0.44444444444444442, 0.33333333333333337, 0.22222222222222232, 0.11111111111111116, 0.0, -0.11111111111111116, -0.2222222222222221, -0.33333333333333326, -0.44444444444444442, -0.55555555555555536, -0.66666666666666652, -0.77777777777777768, -0.88888888888888884, -1.0]


In [ ]:
# parameters = initial_parameters(expected_performance)
# print(parameters)
# stop()

In [ ]:
# cl = [1.359, 1.1, 0.919, 0.782, 0.62, 0.418, 0.388, 0.075, -0.054, -0.056, 
#           -0.059, -0.103, -0.156, -0.394, -0.415,
#           -0.478, -1.377, -1.471, -1.989]
# print(len(expected_performance))
# parameters = initial_parameters(expected_performance)
# g_theta = calc_gradient_user(expected_performance, parameters, cl)
# g_alpha_b = calc_gradient_exc(expected_performance, parameters, cl)
# parameters = update2(parameters, g_alpha_b, g_theta)

# s_loss, mean_loss = sum_loss(expected_performance, parameters, cl)
# print('mean loss is: '+str(mean_loss))
# print('sum loss is: '+str(s_loss))

## training

In [ ]:
# cl = [1.359, 1.1, 0.919, 0.782, 0.62, 0.418, 0.388, 0.075, -0.054, -0.056, 
#           -0.059, -0.103, -0.156, -0.394, -0.415,
#           -0.478, -1.377, -1.471, -1.989]

try:
    parameters = pickle.load(open('../data/irt_parameters.p', 'rb'))
    uid_p, exc_p = parameters
    if with_gamma and 'gamma' not in exc_p:
        exc_p['gamma'] = 1
    if with_m and 'm' not in exc_p:
        exc_p['m'] = 1
    parameters = [uid_p, exc_p]
    
except:
    print('no parameter file')
    parameters = initial_parameters(expected_performance, with_gamma=with_gamma, with_m=with_m)
    print(parameters)

    
pre_mean_loss = 100000
learning_rate = 0.00001
control_factor = 1
best =10000
cnt = 1.0
history_alpha = []
history_b = []
history_theta = []
history_theta_list = [history_theta]
history_alpha_b = [history_alpha, history_b]
mean_loss = 1000

for i in xrange(1000):
    
    g_alpha_b = calc_gradient_exc(expected_performance, parameters, cl, zoom_factor=1, 
                                  regular_term=False, verbose=False)
#     print('here')
#     print(g_alpha_b)
    g_theta = calc_gradient_user(expected_performance, parameters, cl, zoom_factor=100000,
                                 regular_term=False, verbose=False)
    if 'gamma' in g_alpha_b.columns:
        g_con_theta = constraint_theta(all_data, parameters, cl, item_by=['difficulty'], 
                         user_by=['uid', 'day', 'exc_num', 'exc_times'], zoom_factor=1)
        g_con_b = constraint_b(all_data, parameters, cl, item_by=['difficulty'], 
                         user_by=['uid', 'day', 'exc_num', 'exc_times'], zoom_factor=1)
    #     print(1)
    #     print(g_alpha_b)
    #     print(2)
    #     print(g_con_b)
        g_alpha_b = g_alpha_b.set_index(['difficulty']) + g_con_b.set_index(['difficulty'])
        g_theta = g_theta.set_index(['uid', 'day', 'exc_num', 'exc_times'])+g_con_theta.\
            set_index(['uid', 'day', 'exc_num', 'exc_times'])
        g_alpha_b = g_alpha_b.reset_index()
        g_theta = g_theta.reset_index()
    
#     stop
#     kwargs = {"gamma": 0.9, "eta": learning_rate}
#     print(g_alpha_b)
    kwargs = {"mean_sqr_err": mean_loss, 'err_thres':2, 'grad_thres':10**-8, 
              "eta": learning_rate, 'max_step':control_factor*0.0001}
    if 'gamma' in g_alpha_b.columns:
        delta_alpha_b, history_alpha_b = learning_rate_optimizer(gradient=g_alpha_b, history=history_alpha_b, 
                                                 cols=['alpha', 'b', 'gamma'], 
                                                 how=None, **kwargs)
    elif 'm' in g_alpha_b.columns:
        delta_alpha_b, history_alpha_b = learning_rate_optimizer(gradient=g_alpha_b, history=history_alpha_b, 
                                                 cols=['alpha', 'b', 'm'], 
                                                 how=None, **kwargs)
    else:
        delta_alpha_b, history_alpha_b = learning_rate_optimizer(gradient=g_alpha_b, history=history_alpha_b, 
                                                 cols=['alpha', 'b'], 
                                                 how=None, **kwargs)
    kwargs = {"mean_sqr_err": mean_loss, 'err_thres':2, 'grad_thres':10**-8, "eta": 
              learning_rate, 'max_step':control_factor*0.01}
    delta_theta, history_theta_list = learning_rate_optimizer(gradient=g_theta, history=history_theta_list, 
                                             cols=['theta'], 
                                             how=None, **kwargs)
    
    if len(history_alpha)>5:
        del history_alpha[0]
        del history_b[0]
        del history_theta[0]
        
        
    parameters = update2(parameters, delta_theta,  delta_alpha_b)
    if i%100 ==1:
        s_loss, mean_loss = sum_loss(expected_performance, parameters, cl, verbose=False)
        diff = pre_mean_loss - mean_loss
#         if diff<0.01 and diff>0:
#             cnt*=1.1
#         elif diff<0:
#             cnt*=0.7
         
        if mean_loss<best:
            pickle.dump(parameters, open('../data/irt_parameters.p', 'wb'))
            best = mean_loss
        pre_mean_loss = mean_loss
        print('\n\n')
        print('mean loss is: '+str(mean_loss))
        print('sum loss is: '+str(s_loss))
        write_log(mean_loss, s_loss, log_name)
        print('g_alpha_b:')
        print(g_alpha_b)
        print('delta_alpha_b:')
        print(delta_alpha_b)
        print('g_theta:')
        print(g_theta)
        print('delta_theta:')
        print(delta_theta)
        if 'gamma' in g_alpha_b.columns:
            print('g_con_b:')
            print(g_con_b)
            print('g_con_theta:')
            print(g_con_theta)
        print('parameters')
        print(parameters)
        

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2845: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)





mean loss is: 1.54051090477
sum loss is: 2263.01051911
g_alpha_b:
   difficulty     alpha           b           m
0           1  4.485286   -0.350163  -24.967166
1           2  0.815476   44.520393 -106.111743
2           3  1.564150   22.769777 -106.687054
3           4 -1.394425  -42.748428   98.521713
4           5  1.009378   24.070988  -95.497054
5           6  1.361672   27.155640 -109.888714
6           7  0.473550  -23.542992   16.750414
7           8  1.364412   13.482291  -37.127934
8           9  0.405332    0.005926   -0.280346
9          10 -4.714110 -120.262650  246.939557
delta_alpha_b:
   difficulty         alpha             b             m
0           1  1.143850e-05 -3.844787e-09 -7.396539e-06
1           2  3.781039e-07  6.215139e-05 -1.000000e-04
2           3  1.391058e-06  1.625738e-05 -1.000000e-04
3           4 -1.105551e-06 -5.730245e-05  1.000000e-04
4           5  5.792910e-07  1.816857e-05 -1.000000e-04
5           6  1.054227e-06  2.312347e-05 -1.000000e




mean loss is: 1.54245814632
sum loss is: 2265.87101694
g_alpha_b:
   difficulty      alpha           b           m
0           1   1.707887   -0.157583  -24.833575
1           2   4.160374   44.847319 -105.922247
2           3   7.047109   24.560571 -106.437899
3           4  -9.086097  -43.534028   99.582671
4           5   5.258810   25.141463  -95.331031
5           6   6.398900   28.614962 -109.658333
6           7  -0.266374  -23.340942   16.867982
7           8   1.581968   13.518770  -37.126057
8           9   0.403876    0.005904   -0.280315
9          10 -19.826143 -115.328610  252.207679
delta_alpha_b:
   difficulty         alpha             b             m
0           1  5.233235e-07 -7.783236e-10 -7.270358e-06
1           2  3.105396e-06  6.303968e-05 -1.000000e-04
2           3  8.909928e-06  1.890680e-05 -1.000000e-04
3           4 -1.481177e-05 -5.940168e-05  1.000000e-04
4           5  4.961661e-06  1.981172e-05 -1.000000e-04
5           6  7.346202e-06  2.566418e-05




mean loss is: 1.54449378956
sum loss is: 2268.86137686
g_alpha_b:
   difficulty      alpha           b           m
0           1   1.963669   -0.038947  -24.688005
1           2   4.600396   45.221167 -105.686519
2           3   8.041497   26.547553 -106.045134
3           4 -10.151167  -44.746812  100.677294
4           5   5.988287   26.300258  -95.079061
5           6   7.256912   30.216521 -109.308167
6           7  -0.445848  -23.172210   16.993264
7           8   1.834221   13.534649  -37.116598
8           9   0.402870    0.005889   -0.280287
9          10 -26.360266 -114.558067  258.750848
delta_alpha_b:
   difficulty         alpha             b             m
0           1  5.751342e-07 -4.676791e-11 -7.131753e-06
1           2  3.156627e-06  6.304930e-05 -1.000000e-04
2           3  9.645096e-06  2.172930e-05 -1.000000e-04
3           4 -1.536968e-05 -6.173350e-05  1.000000e-04
4           5  5.348573e-06  2.132636e-05 -1.000000e-04
5           6  7.854824e-06  2.815047e-05




mean loss is: 1.54664099412
sum loss is: 2272.01562037
g_alpha_b:
   difficulty      alpha           b           m
0           1   2.142385    0.080442  -24.523912
1           2   4.847416   45.565804 -105.431262
2           3   8.576973   28.370868 -105.606566
3           4 -10.817018  -45.955234  101.806586
4           5   6.356603   27.388470  -94.802424
5           6   7.726109   31.680259 -108.918219
6           7  -0.259104  -23.023452   17.122853
7           8   2.076434   13.538553  -37.097957
8           9   0.401703    0.005872   -0.280240
9          10 -31.344865 -114.404832  265.815681
delta_alpha_b:
   difficulty         alpha             b             m
0           1  6.156807e-07  1.960816e-10 -6.981870e-06
1           2  3.151962e-06  6.291381e-05 -1.000000e-04
2           3  9.867986e-06  2.439008e-05 -1.000000e-04
3           4 -1.569551e-05 -6.399380e-05  1.000000e-04
4           5  5.420141e-06  2.273021e-05 -1.000000e-04
5           6  8.007226e-06  3.041203e-05




mean loss is: 1.54892633959
sum loss is: 2275.37279286
g_alpha_b:
   difficulty      alpha           b           m
0           1   2.332402    0.200101  -24.342164
1           2   5.096015   45.877408 -105.159879
2           3   9.086105   30.004131 -105.136284
3           4 -11.493818  -47.125489  102.972464
4           5   6.703312   28.393536  -94.509389
5           6   8.176127   32.990119 -108.499592
6           7   0.264327  -22.890675   17.257125
7           8   2.306209   13.531791  -37.070858
8           9   0.400236    0.005850   -0.280167
9          10 -36.813630 -113.731533  273.165261
delta_alpha_b:
   difficulty         alpha             b             m
0           1  6.580324e-07  1.196128e-09 -6.823418e-06
1           2  3.141246e-06  6.287476e-05 -1.000000e-04
2           3  9.986104e-06  2.689309e-05 -1.000000e-04
3           4 -1.597972e-05 -6.634227e-05  1.000000e-04
4           5  5.435250e-06  2.408339e-05 -1.000000e-04
5           6  8.086040e-06  3.251220e-05




mean loss is: 1.55136235255
sum loss is: 2278.9512959
g_alpha_b:
   difficulty      alpha           b           m
0           1   2.547056    0.321801  -24.143535
1           2   5.382419   46.157336 -104.874424
2           3   9.655617   31.455270 -104.640550
3           4 -12.262921  -48.245246  104.177015
4           5   7.097408   29.318223  -94.203099
5           6   8.681716   34.154523 -108.057652
6           7   1.061857  -22.766915   17.396760
7           8   2.527231   13.515205  -37.036495
8           9   0.398406    0.005824   -0.280063
9          10 -42.699965 -112.504076  280.820277
delta_alpha_b:
   difficulty         alpha             b             m
0           1  7.027591e-07  3.059768e-09 -6.657039e-06
1           2  3.138229e-06  6.294976e-05 -1.000000e-04
2           3  1.009926e-05  2.923476e-05 -1.000000e-04
3           4 -1.628987e-05 -6.877359e-05  1.000000e-04
4           5  5.456689e-06  2.539732e-05 -1.000000e-04
5           6  8.164711e-06  3.446744e-05 




mean loss is: 1.55396424148
sum loss is: 2282.77347074
g_alpha_b:
   difficulty      alpha           b           m
0           1   2.771196    0.448020  -23.928771
1           2   5.671093   46.409582 -104.576198
2           3  10.226884   32.745314 -104.121906
3           4 -13.041359  -49.314590  105.422113
4           5   7.497119   30.170985  -93.884361
5           6   9.189279   35.191502 -107.594805
6           7   1.889628  -22.630673   17.543580
7           8   2.741940   13.490125  -36.996064
8           9   0.396188    0.005792   -0.279925
9          10 -48.936739 -110.685627  288.802245
delta_alpha_b:
   difficulty         alpha             b             m
0           1  7.502364e-07  5.884677e-09 -6.483452e-06
1           2  3.141935e-06  6.314566e-05 -1.000000e-04
2           3  1.021763e-05  3.143595e-05 -1.000000e-04
3           4 -1.661535e-05 -7.129828e-05  1.000000e-04
4           5  5.491013e-06  2.668746e-05 -9.980494e-05
5           6  8.249480e-06  3.630812e-05




mean loss is: 1.55674811724
sum loss is: 2286.86298422
g_alpha_b:
   difficulty      alpha           b           m
0           1   3.001191    0.580969  -23.698259
1           2   5.954265   46.637126 -104.266566
2           3  10.780593   33.890641 -103.583336
3           4 -13.807764  -50.328591  106.710399
4           5   7.843832   30.945112  -93.555926
5           6   9.682966   36.114965 -107.113743
6           7   2.615305  -22.469877   17.698958
7           8   2.951381   13.458013  -36.950598
8           9   0.393579    0.005755   -0.279755
9          10 -55.453813 -108.237162  297.134847
delta_alpha_b:
   difficulty         alpha             b             m
0           1  8.007444e-07  9.849911e-09 -6.303168e-06
1           2  3.151831e-06  6.347311e-05 -1.000000e-04
2           3  1.033218e-05  3.351859e-05 -1.000000e-04
3           4 -1.694936e-05 -7.391894e-05  1.000000e-04
4           5  5.469695e-06  2.794540e-05 -9.823553e-05
5           6  8.335342e-06  3.806279e-05




mean loss is: 1.55973380115
sum loss is: 2291.24895389
g_alpha_b:
   difficulty      alpha           b           m
0           1   3.235158    0.722665  -23.452515
1           2   6.232851   46.842669 -103.946938
2           3  11.315533   34.905312 -103.027989
3           4 -14.562381  -51.280617  108.044463
4           5   8.167795   31.636129  -93.221395
5           6  10.162555   36.936888 -106.617238
6           7   3.241853  -22.283953   17.863269
7           8   3.156638   13.420301  -36.901046
8           9   0.390605    0.005712   -0.279555
9          10 -62.179379 -105.116129  305.842305
delta_alpha_b:
   difficulty         alpha             b             m
0           1  8.533792e-07  1.521913e-08 -6.116798e-06
1           2  3.167558e-06  6.394385e-05 -1.000000e-04
2           3  1.044001e-05  3.550574e-05 -1.000000e-04
3           4 -1.729083e-05 -7.663409e-05  1.000000e-04
4           5  5.439522e-06  2.916636e-05 -9.664435e-05
5           6  8.420868e-06  3.975906e-05

In [ ]:
uid_p, exc_p = parameters
print(uid_p.shape)
print(uid_p['theta'].describe())
print(uid_p)
print(exc_p[exc_p['alpha']<0])
exc_p

## data analysis without model

In [ ]:
raw_data =  pd.read_csv('../data/step1_clear_data.csv')

with open('../data/parameter/ability_mapper.p', 'rb') as f:
    parameters = pickle.load(f)
print(parameters)

test_col, diff_def, n_class, group, _, if_merge_min, sampleing_rate = get_env_parameters()
test_col_name, test_file_name = get_test_col_name(test_col)
raw_data,_ = ability_level_mapper(raw_data, col=test_col, target_col_name='performance', parameters=parameters) 
raw_data = raw_data.groupby(['uid', 'day', 'exc_num', 'exc_times', 'difficulty']).mean()['performance'].reset_index()
# first_attempt = raw_data.groupby(['uid', 'exc_num']).head(1).reset_index()

raw_data = raw_data.groupby(['uid', 'day', 'exc_num', 'exc_times']).mean()['performance'].reset_index()
 
first_attempt = raw_data.groupby(['uid', 'exc_num']).head(1).reset_index()
last_attempt = raw_data.groupby(['uid', 'exc_num']).tail(1).reset_index()
print(first_attempt.loc[first_attempt.exc_num==3.2])
print(last_attempt.loc[last_attempt.exc_num==3.2])

last_repetition = raw_data.merge(first_attempt, on=['uid', 'day', 'exc_num'], how='inner')
last_repetition = last_repetition.groupby(['uid', 'exc_num']).tail(1)


last_repetition = last_repetition.drop(['index', 'exc_times_x', 'exc_times_y', 'performance_y'], axis=1)
# print(last_repetition.head())
last_repetition.columns = ['day', 'uid', 'exc_num', 'performance']

first_attempt['level'] = 'first attempt'
last_attempt['level'] = 'last attempt on the last training day'
last_repetition['level'] = 'last repetition on the day of the first attempt'

cols = ['exc_num', 'level', 'performance']
df_tmp = pd.concat([first_attempt[cols], last_repetition[cols], last_attempt[cols]])
# print(df_tmp.head())

# draw figures
sns.set(font_scale=2)
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="performance", hue='level', ci=None, data=df_tmp)
# ax.set_title('Average velocity over all participants')
ax.set(xlabel='exercise index')
ax.set(ylabel=test_col_name)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.gca().legend().set_title('')
f.savefig('../data/figure/all/raw_'+test_file_name+'_avg.png')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="performance", hue='level', ci=None, data=df_tmp)
ax.set_title('Average '+test_col_name+' over all participants')
ax.set(xlabel='exercise index')
ax.set(ylabel=test_col_name)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.gca().legend().set_title('')
f.savefig('../data/figure/all/raw_'+test_file_name+'_avg_with_title.png')

# sns.set(font_scale=2)
# for index, group in uid_p.groupby(['uid']):
#     print(group)
#     tmp = group.reset_index(drop=True)
#     sns.set_style('whitegrid')
#     f, ax= plt.subplots(figsize = (14, 10))
#     ax = sns.barplot(x=tmp.index, y='theta', hue='exc_num', ci=None, data=tmp)
#     ax.set(xlabel='index')
#     ax.set(ylabel='performance')
#     plt.gca().legend().set_title('exercises')
#     f.savefig('../data/figure/velocity/theta_velocity_'+str(index)+'.png')
#     plt.show()

In [ ]:
f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="uid", y="performance", capsize=.13, data=raw_data)
# ax.set_title('Average velocity over all exercise for each participant')
ax.set(ylabel=test_col_name)
ax.set(xlabel='participant index')

### correlation between velocity before and after modeling

In [ ]:
tmp = raw_data.merge(uid_p, on=['uid', 'day', 'exc_num', 'exc_times'], how='left')
df_tmp = pd.DataFrame()

for index, group in tmp.groupby(['uid', 'exc_num']):
    correlation = pearsonr(group['performance'], group['theta'])
    if len(group)<3:
        continue
#     print(correlation)
    tmp2 = pd.DataFrame(np.array(list(index)+list(correlation)).reshape(1,-1), 
                        columns=['uid','exc_num', 'correlation', 'p_value'])
    df_tmp = pd.concat([df_tmp, tmp2])
#     print(index)
#     print(len(group))
#     print(correlation)

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="correlation", capsize=.13, data=df_tmp)
# ax.set_title('Average correlation over all participant')
ax.set(ylabel='correlation')
ax.set(xlabel='excercise index')
f.savefig('../data/figure/all/'+test_file_name+'_correlation_before_after_modeling.png')


f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="correlation", capsize=.13, data=df_tmp)
ax.set_title('Average correlation over all participant')
ax.set(ylabel='correlation')
ax.set(xlabel='excercise index')
f.savefig('../data/figure/all/'+test_file_name+'_correlation_before_after_modeling_with_title.png')



### velocity figures

#### comparison between first and last attempt

In [ ]:
# find out first, last attempt and last repetition on the first day
first_attempt = uid_p.groupby(['uid', 'exc_num']).head(1).reset_index()
last_attempt = uid_p.groupby(['uid', 'exc_num']).tail(1).reset_index()
print(first_attempt.loc[first_attempt.exc_num==3.2])
print(last_attempt.loc[last_attempt.exc_num==3.2])

last_repetition = uid_p.merge(first_attempt, on=['uid', 'day', 'exc_num'], how='inner')
last_repetition = last_repetition.groupby(['uid', 'exc_num']).tail(1)

last_repetition = last_repetition.drop(['index', 'exc_times_x', 'exc_times_y', 'theta_y'], axis=1)
last_repetition.columns = ['day', 'uid', 'exc_num', 'theta']

first_attempt['level'] = 'first attempt'
last_attempt['level'] = 'last attempt on the last training day'
last_repetition['level'] = 'last repetition on the day of the first attempt'

cols = ['exc_num', 'level', 'theta']
df_tmp = pd.concat([first_attempt[cols], last_repetition[cols], last_attempt[cols]])
# print(df_tmp.head())

# draw figures
sns.set(font_scale=2)
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="theta", hue='level', ci=None, data=df_tmp)
ax.set_title('Average ability ('+test_col_name+') over all participants')
ax.set(xlabel='exercise index')
ax.set(ylabel='ability ('+test_col_name+')')
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.gca().legend().set_title('')
f.savefig('../data/figure/all/theta_'+test_file_name+'_avg.png')

sns.set(font_scale=2)
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="theta", hue='level', ci=None, data=df_tmp)
ax.set_title('Average ability ('+test_col_name+') over all participants')
ax.set(xlabel='exercise index')
ax.set(ylabel='ability ('+test_col_name+')')
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.gca().legend().set_title('')
f.savefig('../data/figure/all/theta_'+test_file_name+'_avg_with_title.png')



# sns.set(font_scale=2)
# for index, group in uid_p.groupby(['uid']):
#     print(group)
#     tmp = group.reset_index(drop=True)
#     sns.set_style('whitegrid')
#     f, ax= plt.subplots(figsize = (14, 10))
#     ax = sns.barplot(x=tmp.index, y='theta', hue='exc_num', ci=None, data=tmp)
#     ax.set(xlabel='index')
#     ax.set(ylabel='theta')
#     plt.gca().legend().set_title('exercises')
#     f.savefig('../data/figure/velocity/theta_velocity_'+str(index)+'.png')
#     plt.show()

#### variance over all exercises for each person

In [ ]:
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="uid", y="theta", capsize=.13, data=uid_p)
ax.set_title('Average ability over all exercise for each participant')
ax.set(xlabel='participant index')
f.savefig('../data/figure/velocity/theta_velocity_avg_ability_barplot.png')

sns.set_style('whitegrid')
f, ax= plt.subplots(figsize = (14, 10))

ax = sns.boxplot(x="uid", y="theta", data=uid_p)
ax.set_title('The ability of each participant')
ax.set(xlabel='participant index')
# f.savefig('../data/figure/velocity/theta_velocity_avg_ability_boxplot.png')

### average ability over all participants on each training day

In [ ]:
f, ax= plt.subplots(figsize = (14, 10))
ax = sns.barplot(x="day", y="theta", capsize=.13, data=uid_p)
ax.set(xlabel='day')
f.savefig('../data/figure/all/theta_'+test_file_name+'_avg_ability_barplot.png')

f, ax= plt.subplots(figsize = (14, 10))
ax = sns.barplot(x="day", y="theta", capsize=.13, data=uid_p)
ax.set_title('Average ability over all exercise for each participant')
ax.set(xlabel='day')
f.savefig('../data/figure/all/theta_'+test_file_name+'_avg_ability_barplot_with_title.png')

#### deviation figures

In [ ]:
# # find out first, last attempt and last repetition on the first day
# first_attempt = uid_p.groupby(['uid', 'exc_num']).head(1).reset_index()
# last_attempt = uid_p.groupby(['uid', 'exc_num']).tail(1).reset_index()

# last_repetition = uid_p.merge(first_attempt, on=['uid', 'day', 'exc_num'], how='inner')
# last_repetition = last_repetition.groupby(['uid', 'exc_num']).tail(1)

# last_repetition = last_repetition.drop(['index', 'exc_times_x', 'exc_times_y', 'theta_y'], axis=1)
# last_repetition.columns = ['day', 'uid', 'exc_num', 'theta']

# first_attempt['level'] = 'first attempt'
# last_attempt['level'] = 'last attempt on the last training day'
# last_repetition['level'] = 'last repetition on the day of the first attempt'

# cols = ['exc_num', 'level', 'theta']
# df_tmp = pd.concat([first_attempt[cols], last_repetition[cols], last_attempt[cols]])
# print(df_tmp.head())

# # draw figures
# sns.set(font_scale=2)
# sns.set_style('whitegrid')

# f, ax= plt.subplots(figsize = (14, 10))

# ax = sns.barplot(x="exc_num", y="theta", hue='level', ci=None, data=df_tmp)
# ax.set_title('Average ability(deviation) over all participant')
# ax.set(xlabel='exercise index')
# ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
# plt.gca().legend().set_title('')
# f.savefig('../data/figure/deviation/theta_deviation_avg.png')



# sns.set(font_scale=2)
# for index, group in uid_p.groupby(['uid']):
#     print(group)
#     tmp = group.reset_index(drop=True)
#     sns.set_style('whitegrid')
#     f, ax= plt.subplots(figsize = (14, 10))
#     ax = sns.barplot(x=tmp.index, y='theta', hue='exc_num', ci=None, data=tmp)
#     ax.set(xlabel='index')
#     ax.set(ylabel='theta (velocity)')
#     plt.gca().legend().set_title('exercises')
#     f.savefig('../data/figure/deviation/theta_deviation_'+str(index)+'.png')
#     plt.show()

In [ ]:
# sns.set_style('whitegrid')

# f, ax= plt.subplots(figsize = (14, 10))

# ax = sns.barplot(x="uid", y="theta", capsize=.13, data=uid_p)
# ax.set_title('Average ability(deviation) over all exercise for each participant')
# ax.set(xlabel='participant index')
# f.savefig('../data/figure/deviation/theta_deviation_avg_ability_barplot.png')

# sns.set_style('whitegrid')
# f, ax= plt.subplots(figsize = (14, 10))

# ax = sns.boxplot(x="uid", y="theta", data=uid_p)
# ax.set_title('The ability(deviation) of each participant')
# ax.set(xlabel='participant index')
# f.savefig('../data/figure/deviation/theta_deviation_avg_ability_boxplot.png')

## test

### online test

In [ ]:
# test_data = pd.read_csv('../data/step1_clear_data.csv')
# performance_data = pd.read_csv('../data/step2_expected_performance.csv')
# with open('../data/parameter/ability_mapper.p', 'rb') as f:
#     parameters = pickle.load(f)
# test_col, diff_def, n_class, group, _, if_merge_min, sampleing_rate = get_env_parameters()
# test_data,_ = ability_level_mapper(test_data, col=test_col, target_col_name='performance', parameters=parameters) 
# tmp1 = find_segment(test_data, cols=['uid', 'day','exc_num', 'exc_times'], values=[1, 3, 3.2, 1])
# tmp2 = find_segment(test_data, cols=['uid', 'day','exc_num', 'exc_times'], values=[1, 5, 3.2, 2])
# test_data = pd.concat([tmp1, tmp2])
# test_data = test_data.groupby(['uid', 'day', 'exc_num', 'exc_times', 'difficulty']).mean()['performance'].reset_index()
# print(test_data)
# tmp1 = find_segment(all_data, cols=['uid', 'day','exc_num', 'exc_times'], values=[1, 3, 3.2, 1])
# tmp2 = find_segment(all_data, cols=['uid', 'day','exc_num', 'exc_times'], values=[1, 5, 3.2, 2])
# test_data = pd.concat([tmp1, tmp2])
# print(test_data)

### average difference between model processed data and model parameter estimates

In [ ]:
kwargs = {"length": 5000}
print(all_data['exc_num'].unique())
ability = calc_test_theta(all_data, cl, exc_p, user_by=['uid', 'day', 'exc_num', 'exc_times']
                          , item_by=['difficulty'], **kwargs)
result = ability.set_index(['uid', 'day', 'exc_num', 'exc_times']) - \
    uid_p.set_index(['uid', 'day', 'exc_num', 'exc_times'])
result = (result**2).pow(1/2)
result = result.reset_index()
# print('average theta:', result)
# print(result)

sns.set_style('whitegrid')
f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="theta", data=result)
ax.set(xlabel='excercise index')
ax.set(ylabel='ability('+test_col_name+')/theta')
f.savefig('../data/figure/all/difference_processed_and_estimates_'+test_file_name+'_.png')

sns.set_style('whitegrid')
f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="theta", data=result)
ax.set_title('Average difference (model processed data/model parameter estimates)')
ax.set(xlabel='excercise index')
ax.set(ylabel='ability('+test_col_name+')/theta')
f.savefig('../data/figure/all/difference_processed_and_estimates_'+test_file_name+'_with_title.png')

In [ ]:
# tmp = ability.set_index(['uid', 'day', 'exc_num', 'exc_times'])-uid_p.loc[uid_p.uid==1]. \
#     set_index(['uid', 'day', 'exc_num', 'exc_times'])
# tmp = tmp.reset_index()

# tmp.groupby(['exc_num'])['theta'].mean()

In [ ]:
ability = calc_test_theta(test, cl, exc_p, item_by=[], 
                          user_by=['uid', 'day', 'exc_num', 'exc_times', 'difficulty'], **kwargs)
min_perf = ability['theta'].min()
max_perf = ability['theta'].max()
tmp2 = ability.groupby(['uid', 'day', 'exc_num', 'exc_times'])['theta'].std()/(max_perf-min_perf)
tmp2 = tmp2.reset_index()
tmp2['theta'].mean()

In [ ]:
# tmp = all_data.loc[all_data.uid==test_uid]
tmp = test
# print(tmp)
min_perf = tmp['performance'].min()
max_perf = tmp['performance'].max()
tmp1 = tmp.groupby(['uid', 'day', 'exc_num', 'exc_times'])['performance'].std()/(max_perf-min_perf)
tmp1 = tmp1.reset_index()
tmp1['performance'].mean()

In [ ]:
if 'level' not in tmp2.columns:
    tmp2.columns = ['uid', 'day', 'exc_num', 'exc_times', 'diff_std']
    tmp1.columns = ['uid', 'day', 'exc_num', 'exc_times', 'diff_std']
tmp2['level'] = 'with modeling'
tmp1['level'] = 'without modeling'
tmp = pd.concat([tmp1, tmp2])
tmp.to_csv('../data/diff_std_theta_uid8.csv')
# print(tmp)
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="diff_std", hue='level', capsize=.13, data=tmp)
ax.set_title('Standard deviation over difficulties for each exercise on the test data')
ax.set(xlabel='exercies index')
ax.set(ylabel='standard deviation')
plt.gca().legend().set_title('')
f.savefig('../data/figure/velocity/diff_std_theta_barplot.png')


In [ ]:
ability = calc_test_theta(all_data, cl, exc_p, item_by=[], 
                          user_by=['uid', 'day', 'exc_num', 'exc_times', 'difficulty'], **kwargs)
min_perf = ability['theta'].min()
max_perf = ability['theta'].max()
tmp2 = ability.groupby(['uid', 'day', 'exc_num', 'exc_times'])['theta'].std()/(max_perf-min_perf)
tmp2 = tmp2.reset_index()
tmp2['theta'].mean()

In [ ]:
tmp = all_data.loc[all_data.uid!=test_uid]
# print(tmp)
min_perf = tmp['performance'].min()
max_perf = tmp['performance'].max()
tmp1 = tmp.groupby(['uid', 'day', 'exc_num', 'exc_times'])['performance'].std()/(max_perf-min_perf)
tmp1 = tmp1.reset_index()
tmp1['performance'].mean()

In [ ]:
if 'level' not in tmp2.columns:
    tmp2.columns = ['uid', 'day', 'exc_num', 'exc_times', 'diff_std']
    tmp1.columns = ['uid', 'day', 'exc_num', 'exc_times', 'diff_std']
tmp2['level'] = 'with modeling'
tmp1['level'] = 'without modeling'
tmp = pd.concat([tmp1, tmp2])
tmp.to_csv('../data/diff_std_theta_uid8.csv')
# print(tmp)
sns.set_style('whitegrid')

f, ax= plt.subplots(figsize = (14, 10))

ax = sns.barplot(x="exc_num", y="diff_std", hue='level', capsize=.13, data=tmp)
ax.set_title('Standard deviation over difficulties for each exercise on the training data')
ax.set(xlabel='exercies index')
ax.set(ylabel='standard deviation')
plt.gca().legend().set_title('')
f.savefig('../data/figure/velocity/diff_std_theta_train_barplot.png')


### ability estimation for a certain exercise

In [ ]:
# all_data = pd.read_csv('../data/step1_clear_data.csv')
# # performance_data = pd.read_csv('../data/step2_expected_performance.csv')
# selected_uid_day_exc_times = (1, 1, 1.1, 2)
# selected_data = all_data.set_index(['uid', 'day', 'exc_num', 'exc_times']).loc[selected_uid_day_exc_times]   

# test_col, diff_def, n_class, group, _, if_merge_min, sampleing_rate = get_env_parameters()

In [ ]:
# def estimate_ability(data, performance_data, exc_p, test_col, cl):
# #     print(data.head(1))
#     abi = data[['front', 'difficulty']]
# #     print(abi.head())
#     abi['b'] = abi.apply(lambda x: exc_p.loc[exc_p.difficulty==x['difficulty'], 'b'].values[0], axis=1)
#     abi = abi.reset_index()
#     with open('../data/parameter/ability_mapper.p', 'rb') as f:
#         parameters = pickle.load(f)
#     abi,_ = ability_level_mapper(abi, col=test_col, target_col_name=test_col, parameters=parameters)
    
#     def get_abi(x):
#         if x[test_col] == 20:
#             return 4
#         elif x[test_col] ==1:
#             return -4
#         else:
#             return x['b']-(cl[int(x[test_col])-2]+ cl[int(x[test_col]-1)])
                           
                           
#     abi['abi'] = abi.apply(get_abi, axis=1)
#     return abi

def estimate_ability(df, diff_df, test_col, cl, by=['day', 'uid', 'exc_num', 'exc_times']):
    # 1. calculate difficulty for df
    # 1.1 in our proceesed data, there is already difficulty in it
    if 'difficulty' in df.columns:
        
    # 2. calculate performance for df
    # 2.1 load param. from file
        with open('../data/parameter/ability_mapper.p', 'rb') as f:
            parameters = pickle.load(f)
        df, _ = ability_level_mapper(df, col=test_col, target_col_name=test_col, parameters=parameters)
        print(2)
        
    # 3. merge df and diff_df to get parameter of difficulty
        print(len(df))
        df = df.merge(diff_df, on=['difficulty'])
        print(len(df))
        print(3)
        
    # 4. find c from cl with usage of perf. calculate ability according to diff param. and c.
        def get_abi(x):
            if x[test_col] == 20:
                return 4
            elif x[test_col] ==1:
                return -4
            else:
                return x['b']-(cl[int(x[test_col])-2]+ cl[int(x[test_col]-1)])
        
        %timeit df['ability'] = df.apply(get_abi, axis=1)
        print(4)
        
    # summary
    # 1. calc std in each group for ability, and describe them
    print('warning: you use default group unit')
    summary = df.groupby(['day', 'uid', 'exc_num', 'exc_times']).std()['ability']
    print(summary.describe())
    # 2. find out the 5 worst group with large std
    worst = summary.sort_values().tail(5).reset_index()
    worst.columns = ['day', 'uid', 'exc_num', 'exc_times', 'std']
    print(worst)
    del summary
    return df
        

In [ ]:
ability = estimate_ability(all_data ,exc_p, test_col, cl)
# sns.set_style('whitegrid')
# f, ax= plt.subplots(figsize = (14, 10))
# ax = sns.lineplot(x=ability.index, y='abi', data=ability)
# plt.show()

In [ ]:
delta.loc[(delta[col]<grad_thres) & (delta[col]>0), col] + grad_thres

In [ ]:
# tmp = expected_performance.groupby(['exc_num', 'difficulty'])['performance'].mean().reset_index()
# tmp.to_csv('../data/exc_difficulty.csv', index=False)